# Adding Post-Newtonian general relativity corrections

It's easy to add post-newtonian corrections to your REBOUND simulations with REBOUNDx.  Let's start with a simulation without GR:

In [1]:
import rebound
sim = rebound.Simulation()
sim.add(m=1.) # Sun
sim.add(m=1.66013e-07,a=0.387098,e=0.205630) # Mercury-like
sim.move_to_com() # Moves to the center of momentum frame

sim.integrate(10.)
print("pomega = %.16f"%sim.calculate_orbits()[0].pomega)

pomega = 0.0000000000000000


As expected, the pericenter did not move at all.  Now let's add GR

In [2]:
import reboundx
rebx = reboundx.Extras(sim)
rebx.add_gr()

and run for another 100 time units

In [3]:
deltat = 100.
sim.integrate(sim.t + deltat)
print("pomega = %.16f"%sim.calculate_orbits()[0].pomega)
juliancentury = 628.33195 # in yr/2pi
arcsec = 4.8481368e-06 # in rad
print("Rate of change of pomega = %.4f [arcsec / Julian century]"% (sim.calculate_orbits()[0].pomega/deltat*juliancentury/arcsec))

pomega = 0.0000332618518556
Rate of change of pomega = 43.1083 [arcsec / Julian century]


As expected there was pericenter precession. The literature value is 42.98 arcsec / century. 

**Units**

To add GR, you need to pass `add_gr` the speed of light `c` in the units appropriate to the simulation.  By not passing a value above, `c` defaulted to our default units of AU, $M_\odot$ and yr/$2\pi$, for which `G=1` which is what we used for our initial conditions above.

But imagine now we wanted to instead use SI units:

In [4]:
sim = rebound.Simulation()
sim.G = 6.67408e-11
sim.add(m=1.989e30) # Sun
sim.add(m=3.30216458e23,a=5.79090366e10,e=0.205630) # SI units
sim.move_to_com() # Moves to the center of momentum frame
rebx = reboundx.Extras(sim)

If you call `add_gr()` like before you'll get an error telling complaining you changed `G`.  You must explicitly pass `c`:

In [5]:
rebx.add_gr(c=3.e8)

Alternatively, if you set the `units` member in Simulation (see [Units.ipynb](https://github.com/hannorein/rebound/blob/master/ipython_examples/Units.ipynb)), REBOUNDx will use that to calculate the appropriate value of `c`, e.g.,

In [6]:
import rebound
import reboundx
sim = rebound.Simulation()
sim.units = ('m', 's', 'kg')
sim.add(m=1.989e30) # Sun
sim.add(m=3.30216458e23,a=5.79090366e10,e=0.205630) # SI units
sim.move_to_com() # Moves to the center of momentum frame
rebx = reboundx.Extras(sim)
rebx.add_gr()
print("c = {0:e} m/s".format(rebx.gr.c))

c = 2.997804e+08 m/s


**Variants**

Above we always called `add_gr`.  This is the safe default, but there are other options that will make your integrations run faster:

* `add_gr` (default): Allows for multiple massive bodies (Eq. 1 of Benitez and Gallardo 2008).  This gets the GR corrections to the mean motion and the precession correct, and it doesn't matter how you order your planets.  This is the safe option, though it is slower.

* `add_gr_single_mass`: Assumes that only a single mass (`particles`[0]) is the source of post-newtonian corrections (Eq. 2 of Benitez and Gallardo 2008).  This should be good enough for most applications with planets orbiting stars in which GR matters.  Gets both the mean motion and precession correct, **but you must set the central mass as `particles[0]`**.  Will be faster than `add_gr`, particularly with several bodies.

* `add_gr_potential`: This is the simplest potential you can use for GR.  It gets the precession right, but the mean motion is wrong by $\mathcal{O}(GM/ac^2)$.  It is the fastest option, and because it's not velocity-dependent, it automatically keeps WHFAST symplectic.